In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import sys
import csv
import os
from sklearn import metrics
from dateutil.parser import parse
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import lightgbm as lgb
%matplotlib inline

In [2]:
import numpy as np, pandas as pd, os
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import random
import string

In [3]:
data_version_input = "01-10-2020"

In [4]:
df_train = pd.read_csv(f"../../data/kalapa/{data_version_input}/train.csv")
df_test = pd.read_csv(f"../../data/kalapa/{data_version_input}/test.csv")

/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_transform = pd.concat([df_train.iloc[:,2:], df_test.iloc[:,1:]], axis = 0)
data_type = {
    "cat_num":["Field_22","Field_13", "Field_41", "Field_42","Field_50", "Field_51", "Field_53",\
               "Field_73",  "partner0_A", "partner0_B", "partner0_C", "partner0_D", "partner0_E",\
               "partner0_F", "partner0_G", "partner0_H", "partner1_A", "partner1_C", "partner1_G", "partner1_H",\
               "partner2_A", "partner2_C", "partner2_D", "partner2_F", "partner2_H","partner3_A", "partner3_D",\
               "partner3_E", "partner4_L", "partner5_A", "partner5_D", "partner5_G"],
    
    "cat_str" : ["Field_3","Field_4", "Field_12","Field_36", "Field_38", "Field_54", "Field_62","Field_65", \
                 "Field_66", "info_social_sex", "brief", "gioiTinh"], 
    
    "date" : ["Field_1", "Field_2", "Field_5", "Field_6", "Field_7", "Field_8", "Field_9", "Field_11",\
             "Field_15", "Field_25", "Field_32", "Field_33", "Field_35", "Field_40","Field_43", "Field_44", \
              "F_startDate", "F_endDate", "E_startDate", "E_endDate", "C_startDate",
             "C_endDate", "G_startDate", "G_endDate", "A_startDate", "A_endDate", "ngaySinh"],
    
    "text" : ["Field_18","currentLocationCity", "homeTownCity", "Field_48", "Field_49",\
             "currentLocationCountry", 'homeTownState','data.basic_info.locale', "currentLocationName", \
              'homeTownName', "homeTownCountry", "maCv", "currentLocationState", "Field_61", "Field_56",],
    
    "drop" : ["Field_45", "Field_34","Field_68", "diaChi", "Field_46", "Field_55"]
}

df_transform = df_transform.drop(data_type["drop"], axis = 1)
df_transform = df_transform.drop(data_type["text"], axis = 1)
df_transform = df_transform.drop(data_type["date"], axis = 1)
for feature in  data_type["cat_num"]:
    df_transform[feature] = df_transform[feature].fillna("Nan")
    df_transform = pd.get_dummies(df_transform, columns=[feature])
for feature in  data_type["cat_str"]:
    df_transform[feature] = df_transform[feature].fillna("Nan")
    df_transform = pd.get_dummies(df_transform, columns=[feature])
"""    
for feature in data_type["date"]:
    df_transform[feature] = df_transform[feature].map(lambda x: float(parse(str(x)).timestamp()) if type(x) is str else x)
    df_transform[feature] = df_transform[feature].fillna(df_transform[feature].mean())
"""
train_dev = df_transform.iloc[:len(df_train),:]
train_dev = train_dev.fillna(train_dev.mean())

test = df_transform.iloc[len(df_train):,:]
test = test.fillna(test.mean())

scaler = MinMaxScaler()
scaler.fit(train_dev)
X_transform = scaler.transform(train_dev)
X_transform = pd.DataFrame(data = X_transform)

test_transform = scaler.transform(test)

test = pd.DataFrame(data = test_transform)
train_dev = pd.concat([df_train.label, X_transform], axis = 1)
train_dev.reset_index(drop=True,inplace=True)

In [6]:
params = {}
params['learning_rate'] = 0.01
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.1
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10

In [7]:
def ginicof(y, preds):
    fpr, tpr, thresholds = metrics.roc_curve(y, preds, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    ginicof = 2 * auc - 1
    return ginicof

In [8]:
train_dev.iloc[:,1:]

,0,1,2,3,4,5,6,7,8,9,...,300,301,302,303,304,305,306,307,308,309
0,0.200000,0.000000,0.142906,0.750000,0.034483,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.219308,0.038902,0.120912,0.747555,0.067829,0.011264,0.011115,0.090948,0.028086,0.809537,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.200000,0.000000,0.167785,0.750000,0.021552,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.219308,0.038902,0.120912,0.747555,0.067829,0.011264,0.011115,0.090948,0.028086,0.809537,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.219308,0.038902,0.120912,0.747555,0.067829,0.011264,0.011115,0.090948,0.028086,0.809537,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53025,0.200000,0.145688,0.050000,0.750000,0.010776,0.000000,0.000000,0.166667,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
53026,0.219308,0.038902,0.120912,0.747555,0.067829,0.011264,0.011115,0.090948,0.028086,0.809537,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
53027,0.200000,0.000000,0.134732,0.750000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
53028,0.219308,0.038902,0.120912,0.747555,0.067829,0.011264,0.011115,0.090948,0.028086,0.809537,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [9]:
test

,0,1,2,3,4,5,6,7,8,9,...,300,301,302,303,304,305,306,307,308,309
0,0.200000,0.000000,0.050000,0.750000,0.092672,0.000000,0.000000,0.166667,0.000000,1.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.200000,0.000000,0.035000,0.750000,0.051724,0.000000,0.000000,0.166667,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.220199,0.043991,0.122980,0.748262,0.070501,0.009317,0.009205,0.091841,0.025276,0.824283,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.200000,0.000000,0.050000,0.750000,0.000000,0.000000,0.000000,0.166667,0.000000,1.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.220199,0.043991,0.122980,0.748262,0.070501,0.009317,0.009205,0.091841,0.025276,0.824283,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20376,0.220199,0.043991,0.122980,0.748262,0.070501,0.009317,0.009205,0.091841,0.025276,0.824283,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20377,0.220199,0.043991,0.122980,0.748262,0.070501,0.009317,0.009205,0.091841,0.025276,0.824283,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20378,0.200000,0.602564,0.206800,0.750000,0.370690,0.000000,0.000000,0.333333,0.000000,1.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
20379,0.220199,0.043991,0.122980,0.748262,0.070501,0.009317,0.009205,0.091841,0.025276,0.824283,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
ginis  = []
iter_predicttion = []
for i in range(4):
    pred_stack = []
    kf = KFold(n_splits = 8, shuffle=True)
    fold = kf.split(train_dev)
    for train_index, dev_index in fold:
        train1 = train_dev.iloc[train_index,1:]
        dev1 = train_dev.iloc[dev_index,1:]

        d_train = lgb.Dataset(train1, label = train_dev.loc[train_index]["label"])
        clf = lgb.train(params,d_train, 2500)
        predictions_dev = clf.predict(dev1)

        predictions_test = clf.predict(test)
        gini = ginicof(train_dev.loc[dev_index]["label"], predictions_dev)
        pred_stack.append(predictions_test)
        ginis.append(gini)
    predictions = np.asarray(pred_stack)
    predictions = np.mean(predictions, axis = 0)

    iter_predicttion.append(predictions)

In [ ]:
print(iter_predicttion)

In [ ]:
predictions = np.asarray(iter_predicttion)
predictions = np.mean(predictions, axis = 0)
predictions = list(predictions)

for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        predictions[i] = 1
    else:
        predictions[i] = 0
        
new_label = pd.DataFrame(predictions, columns = ["label"])

new_data_id = []
for i in range(len(new_label)):
    idx = len(df_train) + i
    new_data_id.append(idx)
new_data_id = pd.DataFrame(new_data_id, columns = ["id"])

new_data =  pd.concat([new_data_id, new_label, test.iloc[:,1:]], axis = 1)
new_data 

In [ ]:
train_dev = pd.concat([df_train.id, train_dev], axis = 1)

In [ ]:
train_dev

In [ ]:
test = pd.concat([df_test.id, test], axis = 1)

In [ ]:
result_str = ''.join(random.choice(string.ascii_letters) for i in range(5))
data_version = f"06-10-2020-{result_str.lower()}"
print(data_version)

In [ ]:
os.makedirs(f"../../data/kalapa/{data_version}/", exist_ok=True)
train_dev.to_csv(f"../../data/kalapa/{data_version}/train.csv", index = False)
test.to_csv(f"../../data/kalapa/{data_version}/test.csv", index = False)
new_data.to_csv(f"../../data/kalapa/{data_version}/new_data.csv", index = False)